# Feature selection
 
1. Filter methods: ReliefF (classification), correlation (quantitative target)
2. Wrapper methods: Forward selection and backward elimination (e.g. with random forests)
3. Embedded methods: Elastic net (logistic) regression

Let us consider for now two-class classification of species. Furthermore, we will use multi-class classification of tissues for some exercises.

______________________________

## Embedded Methods 
Embedded Methods embeds the feature selection process inside the ML models. They learn the best feature subset while the model is being created. So compared with the Filter Methods, they tend to have better performance. Compared with the Wrapper Methods, they save large processing time and computing power.

### Regularization Based Methods

Many ML models introduce penalties (L1 norm or L2 norm) in their loss functions to prevent the overfitting problem. The L1 norm penalization in linear models (such as Linear SVC, Logistic Regression, Linear Regression) tends to shrink the feature coefficients of some features to zero therefore results in sparse solutions. So we can assign scores to each feature by their (absolute values of) coefficients in the linear model with regularization. The higher the coefficients, the more important the features in a linear model.